# Generate Test Data

In [1]:
from getpass import getuser
from pathlib import Path

import dask
import intake
import numpy as np
import xarray as xr

import marEx
import marEx.helper as hpc

In [2]:
# Lustre Scratch Directory
scratch_dir = Path("/scratch") / getuser()[0] / getuser()
save_dir = Path("/home") / getuser()[0] / getuser() / "opt" / "marEx" / "tests" / "data"

In [3]:
# Start Dask Cluster
client = hpc.start_local_cluster(
    n_workers=16, threads_per_worker=1, scratch_dir=scratch_dir / "clients"
)  # Specify temporary scratch directory for dask to use

Hostname: l20447
Forward Port: l20447:8787
Dashboard Link: localhost:8787/status


# Raw SST Data

In [ ]:
cat = intake.open_catalog("https://raw.githubusercontent.com/eerie-project/intake_catalogues/main/eerie.yaml")

In [ ]:
# Import 40 years of OSTIA Observations
dat_regrid = cat["dkrz.disk.observations.OSTIA.OSTIA.daily"]
sst_regrid = dat_regrid(chunks={}).to_dask().sst.astype(np.float32).coarsen(lat=5, lon=5).mean().rename("to")
sst_regrid

In [ ]:
# Import 40 years of Daily ICON data (ref. EERIE project)
dat_native = cat["dkrz.disk.model-output.icon-esm-er.eerie-control-1950.v20240618"]["ocean"]["native"]
sst_native = (
    dat_native["2d_daily_mean"](chunks={}).to_dask().to.isel(depth=0).drop_vars({"depth", "cell_sea_land_mask"}).chunk({"time": 32})
)
sst_native

In [ ]:
# Define subset
time_slice_ostia = slice("1982-01-01", "2022-01-01")
time_slice_icon = slice("1991-01-01", "2031-01-01")
regrid_slice_ostia = dict(lat=slice(35, 40), lon=slice(-40, -30))
native_slice_icon = dict(lat=slice(39, 40), lon=slice(-40, -39))

In [ ]:
sst_regrid_slice = sst_regrid.sel(time=time_slice_ostia, **regrid_slice_ostia).chunk({"time": 30, "lat": -1, "lon": -1})
sst_regrid_slice

In [ ]:
# sst_native_slice = sst_native.sel(time=time_slice_icon).sel(**native_slice_icon).drop_vars({"lat", "lon"})
sst_native_slice = sst_native.sel(time=time_slice_icon)
sst_native_slice = sst_native_slice.where(
    ((sst_native_slice.lat.compute() > 39) & (sst_native_slice.lat.compute() < 40))
    & ((sst_native_slice.lon.compute() > -40) & (sst_native_slice.lon.compute() < -39)),
    drop=True,
).drop_vars({"lat", "lon"})
sst_native_slice

In [ ]:
# Save to Zarr
# sst_regrid_slice.to_zarr(save_dir / "sst_gridded.zarr", mode="w")
sst_native_slice.to_zarr(save_dir / "sst_unstructured.zarr", mode="w")

# Pre-processed Data

In [ ]:
time_slice = slice("2010-01-01", "2010-02-01")
extremes_ds = (
    xr.open_zarr(scratch_dir / "mhws" / "extremes_binary_gridded_shifting_hobday.zarr", chunks={})
    .sel(time=time_slice)
    .drop_vars({"thresholds", "dat_anomaly", "dayofyear"})
)
extremes_ds

In [ ]:
# Coarsen extremes_ds (lat & lon)
extremes_ds_coarsen = extremes_ds.coarsen(lat=4, lon=4, boundary="trim").any().chunk({"time": 2, "lat": -1, "lon": -1}).persist()
extremes_ds_coarsen

In [ ]:
extremes_ds_coarsen["lat"] = extremes_ds_coarsen["lat"].astype(np.float32)
extremes_ds_coarsen["lon"] = extremes_ds_coarsen["lon"].astype(np.float32)

In [ ]:
# Clear encoding so we can write to Zarr
extremes_ds_coarsen.encoding = {}
for var in extremes_ds_coarsen.data_vars:
    extremes_ds_coarsen[var].encoding = {}

In [ ]:
extremes_ds_coarsen.to_zarr(save_dir / "extremes_gridded.zarr", mode="w")

### Make version with artificial blob

In [ ]:
extremes_ds_coarsen_blob = extremes_ds_coarsen.copy(deep=True)
extremes_ds_coarsen_blob

In [ ]:
rate = 3  # degrees east per day
start_lon = 170

delta_t = extremes_ds_coarsen_blob.time - extremes_ds_coarsen_blob.time.min()
delta_t = delta_t.dt.days
delta_t

offset_east = delta_t * rate

centroid_lat = delta_t * 0.0  # No movement in latitude
centroid_lon = start_lon + offset_east

In [ ]:
# For each time, create the square blob
blob_size = 7  # degrees

lon_0_360 = xr.where(extremes_ds_coarsen_blob.lon >= 0, extremes_ds_coarsen_blob.lon, extremes_ds_coarsen_blob.lon + 360)

extreme_events_blob = np.zeros_like(extremes_ds_coarsen_blob["extreme_events"].compute().values, dtype=bool)

for i in range(len(extremes_ds_coarsen_blob.time)):
    # Create a square blob
    lat_min = centroid_lat[i] - blob_size / 2
    lat_max = centroid_lat[i] + blob_size / 2
    lon_min = centroid_lon[i] - blob_size / 2
    lon_max = centroid_lon[i] + blob_size / 2

    # Create the mask, accounting for the range of lon from [-180, 180]
    mask = (
        (extremes_ds_coarsen_blob.lat >= lat_min)
        & (extremes_ds_coarsen_blob.lat <= lat_max)
        & (lon_0_360 >= lon_min)
        & (lon_0_360 <= lon_max)
    )

    # Set the mask to True for the blob
    extreme_events_blob[i, mask] = True

# Assign the modified blob back to the dataset
extremes_ds_coarsen_blob["extreme_events"] = (("time", "lat", "lon"), extreme_events_blob)

In [ ]:
extremes_ds_coarsen_blob

In [ ]:
extremes_ds_coarsen_blob.to_zarr(save_dir / "extremes_gridded_blob.zarr", mode="w")

# Make Pre-processed Native Data

In [4]:
cat = intake.open_catalog("https://raw.githubusercontent.com/eerie-project/intake_catalogues/main/eerie.yaml")

In [5]:
# Import 40 years of Daily ICON data (ref. EERIE project)
dat_native = cat["dkrz.disk.model-output.icon-esm-er.eerie-control-1950.v20240618"]["ocean"]["native"]
sst_native = (
    dat_native["2d_daily_mean"](chunks={}).to_dask().to.isel(depth=0).drop_vars({"depth", "cell_sea_land_mask"}).chunk({"time": 32})
)
sst_native

<xarray.DataArray 'to' (time: 18262, ncells: 14886338)> Size: 1TB
dask.array<rechunk-merge, shape=(18262, 14886338), dtype=float32, chunksize=(32, 14886338), chunktype=numpy.ndarray>
Coordinates:
    lat      (ncells) float64 119MB dask.array<chunksize=(14886338,), meta=np.ndarray>
    lon      (ncells) float64 119MB dask.array<chunksize=(14886338,), meta=np.ndarray>
  * time     (time) datetime64[ns] 146kB 1991-01-01T23:59:59 ... 2040-12-30T2...
Dimensions without coordinates: ncells
Attributes:
    CDI_grid_type:                unstructured
    long_name:                    sea water potential temperature
    number_of_grid_in_reference:  1
    param:                        18.4.10
    standard_name:                sea_water_potential_temperature
    units:                        C

In [6]:
time_slice_icon = slice("2000-01-01", "2002-01-01")

In [7]:
sst_native_slice = sst_native.sel(time=time_slice_icon)  # .isel(ncells=native_slice_icon)
sst_native_slice

<xarray.DataArray 'to' (time: 732, ncells: 14886338)> Size: 44GB
dask.array<getitem, shape=(732, 14886338), dtype=float32, chunksize=(32, 14886338), chunktype=numpy.ndarray>
Coordinates:
    lat      (ncells) float64 119MB dask.array<chunksize=(14886338,), meta=np.ndarray>
    lon      (ncells) float64 119MB dask.array<chunksize=(14886338,), meta=np.ndarray>
  * time     (time) datetime64[ns] 6kB 2000-01-01T23:59:59 ... 2002-01-01T23:...
Dimensions without coordinates: ncells
Attributes:
    CDI_grid_type:                unstructured
    long_name:                    sea water potential temperature
    number_of_grid_in_reference:  1
    param:                        18.4.10
    standard_name:                sea_water_potential_temperature
    units:                        C

In [8]:
grid2d = dat_native["2d_grid"](chunks={}).to_dask().rename({"cell": "ncells"})
neighbours = grid2d.neighbor_cell_index.rename({"clat": "lat", "clon": "lon"})
areas = grid2d.cell_area.rename({"clat": "lat", "clon": "lon"})

/home/b/b382615/opt/anaconda3/envs/py310/lib/python3.10/site-packages/intake/readers/readers.py:1327: UserWarning: The specified chunks separate the stored chunks along dimension "nc" starting at index 1. This could degrade performance. Instead, consider rechunking after loading.
  return open_dataset(data.url, **kw)
/home/b/b382615/opt/anaconda3/envs/py310/lib/python3.10/site-packages/intake/readers/readers.py:1327: UserWarning: The specified chunks separate the stored chunks along dimension "edge" starting at index 16777214. This could degrade performance. Instead, consider rechunking after loading.
  return open_dataset(data.url, **kw)
/home/b/b382615/opt/anaconda3/envs/py310/lib/python3.10/site-packages/intake/readers/readers.py:1327: UserWarning: The specified chunks separate the stored chunks along dimension "cell" starting at index 8388606. This could degrade performance. Instead, consider rechunking after loading.
  return open_dataset(data.url, **kw)
/home/b/b382615/opt/anacon

In [9]:
sst_native_slice = sst_native_slice.chunk({"ncells": 100000})
sst_native_slice

<xarray.DataArray 'to' (time: 732, ncells: 14886338)> Size: 44GB
dask.array<rechunk-merge, shape=(732, 14886338), dtype=float32, chunksize=(32, 100000), chunktype=numpy.ndarray>
Coordinates:
    lat      (ncells) float64 119MB dask.array<chunksize=(100000,), meta=np.ndarray>
    lon      (ncells) float64 119MB dask.array<chunksize=(100000,), meta=np.ndarray>
  * time     (time) datetime64[ns] 6kB 2000-01-01T23:59:59 ... 2002-01-01T23:...
Dimensions without coordinates: ncells
Attributes:
    CDI_grid_type:                unstructured
    long_name:                    sea water potential temperature
    number_of_grid_in_reference:  1
    param:                        18.4.10
    standard_name:                sea_water_potential_temperature
    units:                        C

In [ ]:
# Process Data using `MarEx Detect` helper functions:

extremes_ds = marEx.preprocess_data(
    sst_native_slice,
    threshold_percentile=95,  # Use the 95th percentile as the extremes threshold
    dask_chunks={
        "time": 2
    },  # Dask chunks for *output* data (this is much smaller than the input chunks because the Tracking/ID is more memory-intensive)
    neighbours=neighbours,  # Pass information about neighbours to be used in subsequent processing
    cell_areas=areas,  # Pass information about each Unstructured Grid's cell area (in metres) to be used in subsequent processing
    dimensions={"time": "time", "xdim": "ncells"},
)  # Not specifying 'ydim' tells MarEx-Detect that it is an Unstructured Grid
extremes_ds

In [10]:
N_cells = 1000
native_slice_icon = slice(0, N_cells)
extremes_subset = xr.load_dataset(scratch_dir / "mhws" / "extremes_unstructured_small.zarr", chunks={}).isel(
    ncells=native_slice_icon
)
extremes_subset

<xarray.Dataset> Size: 4MB
Dimensions:         (ncells: 1000, time: 732, nv: 3)
Coordinates:
    lat             (ncells) float64 8kB 72.14 72.12 72.17 ... 72.79 72.8 72.78
    lon             (ncells) float64 8kB 73.0 73.12 73.0 ... 78.67 78.43 78.06
  * nv              (nv) int64 24B 0 1 2
  * time            (time) datetime64[ns] 6kB 2000-01-01T23:59:59 ... 2002-01...
Dimensions without coordinates: ncells
Data variables:
    cell_areas      (ncells) float32 4kB 2.429e+07 2.429e+07 ... 2.421e+07
    dat_anomaly     (time, ncells) float32 3MB 1.37 1.293 ... -0.142 -0.2831
    extreme_events  (time, ncells) bool 732kB False False False ... False False
    mask            (ncells) bool 1kB True True True True ... True True True
    neighbours      (nv, ncells) int32 12kB 2 16 8 12 6 ... 999 996 996 996 1003
    thresholds      (ncells) float64 8kB 2.259 2.067 2.322 ... 1.403 1.589
Attributes:
    detrend_orders:        [1]
    exact_percentile:      False
    force_zero_mean:       True
    method_anomaly:        detrended_baseline
    method_extreme:        global_extreme
    preprocessing_steps:   ['Removed polynomial trend orders=[1] & seasonal c...
    std_normalise:         False
    threshold_percentile:  95

In [ ]:
# extremes_subset = xr.load_dataset(scratch_dir / "mhws" / "extremes_unstructured_small.zarr", chunks={})

# extremes_subset = extremes_subset.where(
#     ((extremes_subset.lat.compute() > 39) & (extremes_subset.lat.compute() < 40)) &
#     ((extremes_subset.lon.compute() > -40) & (extremes_subset.lon.compute() < -39)),
#     drop=True
# )
# extremes_subset

In [11]:
# Set all neighbours > N_cells to be 0
extremes_subset["neighbours"] = extremes_subset.neighbours.where(extremes_subset.neighbours <= N_cells, 0)

In [ ]:
# # For each value in `extremes_subset.neighbours`, set the value to 0 if the value does not exist in extremes_subset.ncells
# extremes_subset["neighbours"] = extremes_subset.neighbours.where(extremes_subset.neighbours.isin(extremes_subset.ncells+1), 0)

In [12]:
extremes_subset

<xarray.Dataset> Size: 4MB
Dimensions:         (ncells: 1000, time: 732, nv: 3)
Coordinates:
    lat             (ncells) float64 8kB 72.14 72.12 72.17 ... 72.79 72.8 72.78
    lon             (ncells) float64 8kB 73.0 73.12 73.0 ... 78.67 78.43 78.06
  * nv              (nv) int64 24B 0 1 2
  * time            (time) datetime64[ns] 6kB 2000-01-01T23:59:59 ... 2002-01...
Dimensions without coordinates: ncells
Data variables:
    cell_areas      (ncells) float32 4kB 2.429e+07 2.429e+07 ... 2.421e+07
    dat_anomaly     (time, ncells) float32 3MB 1.37 1.293 ... -0.142 -0.2831
    extreme_events  (time, ncells) bool 732kB False False False ... False False
    mask            (ncells) bool 1kB True True True True ... True True True
    neighbours      (nv, ncells) int32 12kB 2 16 8 12 6 2 ... 999 996 996 996 0
    thresholds      (ncells) float64 8kB 2.259 2.067 2.322 ... 1.403 1.589
Attributes:
    detrend_orders:        [1]
    exact_percentile:      False
    force_zero_mean:       True
    method_anomaly:        detrended_baseline
    method_extreme:        global_extreme
    preprocessing_steps:   ['Removed polynomial trend orders=[1] & seasonal c...
    std_normalise:         False
    threshold_percentile:  95

In [13]:
extremes_subset["lat"] = extremes_subset["lat"].astype(np.float32)
extremes_subset["lon"] = extremes_subset["lon"].astype(np.float32)
extremes_subset["thresholds"] = extremes_subset["thresholds"].astype(np.float32)
extremes_subset

<xarray.Dataset> Size: 4MB
Dimensions:         (ncells: 1000, time: 732, nv: 3)
Coordinates:
    lat             (ncells) float32 4kB 72.14 72.12 72.17 ... 72.79 72.8 72.78
    lon             (ncells) float32 4kB 73.0 73.12 73.0 ... 78.67 78.43 78.06
  * nv              (nv) int64 24B 0 1 2
  * time            (time) datetime64[ns] 6kB 2000-01-01T23:59:59 ... 2002-01...
Dimensions without coordinates: ncells
Data variables:
    cell_areas      (ncells) float32 4kB 2.429e+07 2.429e+07 ... 2.421e+07
    dat_anomaly     (time, ncells) float32 3MB 1.37 1.293 ... -0.142 -0.2831
    extreme_events  (time, ncells) bool 732kB False False False ... False False
    mask            (ncells) bool 1kB True True True True ... True True True
    neighbours      (nv, ncells) int32 12kB 2 16 8 12 6 2 ... 999 996 996 996 0
    thresholds      (ncells) float32 4kB 2.259 2.067 2.322 ... 1.403 1.589
Attributes:
    detrend_orders:        [1]
    exact_percentile:      False
    force_zero_mean:       True
    method_anomaly:        detrended_baseline
    method_extreme:        global_extreme
    preprocessing_steps:   ['Removed polynomial trend orders=[1] & seasonal c...
    std_normalise:         False
    threshold_percentile:  95

In [14]:
extremes_subset.to_zarr(save_dir / "extremes_unstructured.zarr", mode="w")